In [4]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_name = 'all_parts'

data_file = f'../{data_name}_vectors_drags_lifts.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

Loaded data from: ../all_parts_vectors_drags_lifts.csv | Columns = 516 / 516 | Rows = 1047 -> 1047


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_505,dim_506,dim_507,dim_508,dim_509,dim_510,dim_511,dim_512,drag,lift
406,-1.334886,0.321034,0.117262,-0.305855,0.444947,1.023753,-0.451729,-1.019958,-0.258471,0.228218,...,2.593884,0.411334,-0.664937,-0.254676,-1.019561,-1.698375,2.505520,-1.301861,0.395,0.425
54,-0.109114,-0.027087,-0.042780,-0.158241,0.175974,-0.289997,-0.133499,0.108182,-0.338404,-0.450839,...,0.087662,1.532475,-0.305243,-0.534035,-0.571023,0.098019,-0.941778,-1.245735,0.645,0.885
241,-0.101703,0.144499,-0.023952,-0.053210,0.061378,-0.311319,0.125281,-0.027643,0.149108,0.443060,...,1.002401,1.859596,-0.137056,0.355251,2.653296,-2.105825,0.281476,-0.948454,0.446,0.756
952,-0.226892,-0.005032,-0.167621,0.388806,0.103854,-0.436283,0.040336,0.165606,0.180177,-0.430575,...,1.288276,0.013573,-2.236531,-0.609770,0.127713,-0.095409,-0.348951,1.393792,0.384,0.177
337,0.185246,-0.069290,0.026386,-0.056042,-0.145993,0.035420,-0.000915,0.042858,0.043498,0.346347,...,0.753645,2.592806,-2.071527,2.080894,-0.331741,-0.813147,-0.123172,-1.497938,0.381,0.074


In [16]:
#save train_df to a csv file
train_df.to_csv('./train_df.csv', index=False)


In [17]:
#save test_df to a csv file
test_df.to_csv('./test_df.csv', index=False)

In [5]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    837.000000
mean       0.445233
std        0.112500
min        0.051000
25%        0.369000
50%        0.427000
75%        0.513000
max        0.944000
Name: drag, dtype: float64


In [6]:
import os
save_path = f'./agModels-{data_name}_{label}'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-all_parts_drag/learner.pkl
Saving ./agModels-all_parts_drag/predi

In [7]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_504,dim_505,dim_506,dim_507,dim_508,dim_509,dim_510,dim_511,dim_512,lift
1046,1.619915,-0.103578,0.073575,-1.121774,0.141612,0.590395,-0.228947,-0.219177,-0.779382,-0.057121,...,0.823198,1.352653,-0.688813,0.550425,-2.302893,0.768312,-1.411428,0.085539,-0.680079,0.346
657,0.271968,-0.005573,-0.117678,-0.265639,-0.109453,-0.103864,0.155861,-0.004339,0.145107,-0.075331,...,2.078918,1.211526,-0.097052,-0.075910,0.071608,0.316292,-1.772259,0.027680,1.245017,0.654
127,0.463927,-0.125690,-0.007975,0.209698,0.010536,0.418671,-0.035722,-0.130384,-0.453760,0.582740,...,1.008988,2.737970,0.340102,0.494566,0.096378,0.921504,0.511855,0.058074,0.371419,0.052
951,0.851177,-0.137168,-0.108522,0.558352,0.014424,-0.032274,0.004961,-0.087621,0.066688,-0.268336,...,2.797498,1.963243,0.650137,-0.393522,1.317100,-0.996707,0.200376,0.762205,0.870829,0.604
134,0.028853,0.104246,-0.015825,-0.181903,-0.039073,-0.342067,0.125156,0.060900,0.152334,-0.304059,...,0.934917,1.027455,0.964021,-1.136593,-0.250099,0.149763,-0.103794,-1.079714,2.322213,0.199


In [8]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_all_parts.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-all_parts_drag/predictor.pkl
Loading: ./agModels-all_parts_drag/learner.pkl
Loading: ./agModels-all_parts_drag/models/trainer.pkl
Loading: ./agModels-all_parts_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/WeightedEnsemble_L2/model.pkl
Evaluation: root_mean_squared_error on test data: -0.08133516600477746
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.08133516600477746,
    "mean_squared_error": -0.006615409229024706,
    "mean_absolute_error": -0.057344734694844204,
    "r2": 0.483051235935815,
    "pearsonr": 0.6989938886459746,
    "median_absolute_error": -0.04067301750183108
}
Loading

0.47504234313964844
0.32410183548927307
0.43337178230285645
0.4535171091556549
0.4726034998893738
0.5108012557029724
0.35632026195526123
0.3872663974761963
0.3801206648349762
0.40799474716186523
0.541803240776062
0.5053961277008057
0.41193848848342896
0.3610454201698303
0.6252260208129883
0.5616629719734192
0.3433525860309601
0.5271213054656982
0.572890043258667
0.5020303726196289
0.4636004567146301
0.4113825559616089
0.5373338460922241
0.449301540851593
0.41023552417755127
0.4241964519023895
0.5753346085548401
0.3983362913131714
0.4407493770122528
0.3461119532585144
0.42644479870796204
0.5155726075172424
0.43160754442214966
0.5375111103057861
0.4602627158164978
0.3792031705379486
0.3873908817768097
0.4007530212402344
0.3662152886390686
0.43104228377342224
0.4331694543361664
0.34225016832351685
0.5971861481666565
0.42466485500335693
0.5496113300323486
0.37409839034080505
0.4155435562133789
0.4562925696372986
0.38299912214279175
0.33838561177253723
0.5291051864624023
0.4226836860179901


Loading: ./agModels-all_parts_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBM_BAG_L2/model.pkl

                     model  score_test  score_val  pred_time_test  \
0    NeuralNetTorch_BAG_L2   -0.079759  -0.077553        9.913691   
1          CatBoost_BAG_L2   -0.079957  -0.077002        9.814588   
2          CatBoost_BAG_L3   -0.080132  -0.077161       18.137650   
3        LightGBMXT_BAG_L2   -0.080963  -0.077242       11.404666   
4      WeightedEnsemble_L3   -0.081098  -0.075372       13.105716   
5          CatBoost_BAG_L4   -0.081155  -0.078054       25.733037   
6      WeightedEnsemble_L2   -0.081335  -0.075364        3.419925   
7    NeuralNetTorch_BAG_L3   -0.081378  -0.076300       18.279189   
8      WeightedEnsemble_L4   -0.081602  -0.075523       18.967369   
9    NeuralNetTorch_BAG_L4   -0.081869  -0.078821       25.939156   
10       LightGBMXT_BAG_L3   -0.081895  -0.077553       19.679989   
11  RandomForestMSE_BAG_L2   -0.082002  -0.077303        9.685664   
12   NeuralNetTorch_BAG_L1   -0.082101  -0.077404        0.563124   
13     WeightedEnsemble_L5   -0.08

In [9]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 513 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [10]:
train_data_pred = predictor.predict(train_data)
test_data_pred = predictor.predict(test_data)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt(f'./{data_name}_vectors_y_test_hat_{label}.csv', test_data_pred, delimiter=',')
np.savetxt(f'./{data_name}_vectors_y_train_hat_{label}.csv', train_data_pred, delimiter=',')

Loading: ./agModels-all_parts_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-all_parts_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts_drag/models/WeightedEnsemble_L2/model.pkl
